In [7]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json
import os
from cassandra.query import dict_factory
import pandas as pd

In [8]:
def connect_db():
    try:
      logging.info("Initiating the database connection procedure")

      # This secure connect bundle is autogenerated when you download your SCB, 
      # if yours is different update the file name below
      cloud_config= {
        'secure_connect_bundle': './secure-connect-thyroid-detection-db.zip'
      }


      # This token JSON file is autogenerated when you download your token, 
      # if yours is different update the file name below
      with open(os.path.join("thyroid_detection_db_token.json")) as f:
          secrets = json.load(f)

      CLIENT_ID = secrets["clientId"]
      CLIENT_SECRET = secrets["secret"]

      auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
      cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
      session = cluster.connect()

      row= session.execute("select release_version from system.local").one()

      if row: 
        print("Connection established...")
        logging.info("Database connection established")

      else:
        try:
            pass
        except Exception as e:
            logging.info("Database connection failed to establish")
            raise CustomException(e,sys)
      
      return session
    
    except Exception as e:
      raise CustomException(e,sys)

In [12]:
def read_table():
    keyspace='thyroid_disease'
    table_name='raw_data'
    logging.info("Initiaing reading data table")
    try:
      session=connect_db()
      session.row_factory=dict_factory
      row=session.execute(f"""SELECT * FROM {keyspace}.{table_name}""")
      logging.info("Data reading completed successfully")
      return row
    except Exception as e:
      logging.info("Data reading unsuccessful")
      raise CustomException(e,sys)

In [14]:
data=read_table()[::]
dataframe=pd.DataFrame(data,).iloc[:,1:]
df_cols=dataframe.columns

Connection established...


In [15]:
dataframe.head()

,age,disease,fti,fti_measured,goitre,hypopituitary,i131_treatment,lithium,on_antithyroid_medication,on_thyroxine,...,t4u,t4u_measured,tbg,tbg_measured,thyroid_surgery,tsh,tsh_measured,tt4,tt4_measured,tumor
0,63,negative.|3080\n,97,t,f,f,f,f,f,f,...,1.03,t,?,f,f,0.88,t,101,t,f
1,49,negative.|947\n,88,t,f,f,f,f,f,f,...,0.91,t,?,f,f,1.9,t,81,t,f
2,45,negative.|2679\n,63,t,f,f,f,f,f,f,...,0.85,t,?,f,f,5,t,54,t,f
3,67,negative.|94\n,121,t,f,f,f,f,f,f,...,0.98,t,?,f,f,2.1,t,119,t,f
4,58,negative.|2568\n,85,t,f,f,f,f,f,f,...,0.88,t,?,f,f,0.17,t,75,t,f
